In [1]:
# test full training of maskcnn_polished_with_rcnn_k_bl
from os.path import join

import numpy as np

import torch

from thesis_v2 import dir_dict

from thesis_v2.data.prepared.yuanyuan_8k import get_data

from thesis_v2.training_extra.maskcnn_like.opt import get_maskcnn_v1_opt_config
from thesis_v2.training_extra.maskcnn_like.training import (train_one,
                                                            partial)

from thesis_v2.models.maskcnn_polished_with_rcnn_k_bl.builder import (
    gen_maskcnn_polished_with_rcnn_k_bl,
    load_modules
)

In [2]:
def train_one_maskcnn_polished_with_rcnn_k_bl(
    split_seed,
    model_seed,
    act_fn,
    loss_type,
    input_size,
    out_channel,
    num_layer,
    kernel_size_l1,
    pooling_ksize,
    scale, scale_name,
    smoothness, smoothness_name,
    pooling_type,
    n_timesteps,
):
    
    load_modules()
    datasets = get_data('a', 200, input_size, ('042318', '043018', '051018'), scale=0.5,
                        seed=split_seed)

    datasets = {
        'X_train': datasets[0].astype(np.float32),
        'y_train': datasets[1],
        'X_val': datasets[2].astype(np.float32),
        'y_val': datasets[3],
        'X_test': datasets[4].astype(np.float32),
        'y_test': datasets[5],
    }

    def gen_cnn_partial(input_size_cnn, n):
        return gen_maskcnn_polished_with_rcnn_k_bl(
                                    input_size=input_size_cnn,
                                    num_neuron=n,
                                    out_channel=out_channel,
                                    kernel_size_l1=kernel_size_l1,  # (try 5,9,13)
                                    kernel_size_l23=3,
                                    act_fn=act_fn,
                                    pooling_ksize=pooling_ksize,  # (try, 1,3,5,7)
                                    pooling_type=pooling_type,  # try (avg, max)  # looks that max works well here?
                                    num_layer=num_layer,
                                    n_timesteps=n_timesteps,
                                    factored_constraint=None,
                                    blstack_pool_ksize=1,
                                    blstack_pool_type=None,
                                    acc_mode='cummean',
                                    bn_after_fc=False,
                                    )

    opt_config_partial = partial(get_maskcnn_v1_opt_config,
                                 scale=scale,
                                 smoothness=smoothness,
                                 group=0.0,
                                 loss_type=loss_type,
                                 )
    
    result = train_one(
        arch_json_partial=gen_cnn_partial,
        opt_config_partial=opt_config_partial,
        datasets=datasets,
        key=f'debug/test_full_training_maskcnn_polished_with_rcnn_k_bl/regular/{model_seed}',
        show_every=100,
        max_epoch=40000,
        model_seed=model_seed,
        return_model=False,
        extra_params={
            # reduce on batch axis
            'eval_fn': {
                'yhat_reduce_axis': 1,
            }
        }
    )
    
    return result['stats_best']['stats']['test']['corr_mean']

In [3]:
maskcnn_param_template = {
    'out_channel': 16,
    'num_layer': 3,
    'kernel_size_l1': 9,
    'pooling_ksize': 3,
    'pooling_type': 'avg',
    'model_seed': 0,
    'split_seed': 'legacy',
}

maskcnn_param_regular = {
    **maskcnn_param_template,
    **{
        'act_fn': 'relu',
        'loss_type': 'mse',
        'smoothness': 0.000005,
        'smoothness_name': '0.000005',
        'scale': 0.01,
        'scale_name': '0.01',
        'input_size': 50,
        'n_timesteps': 4,
    }
}

print(train_one_maskcnn_polished_with_rcnn_k_bl(**maskcnn_param_regular))

{'final_act', 'bn_output', 'fc', 'pooling'}
['bl_stack.layer_list.0.b_conv', 'bl_stack.layer_list.1.b_conv', 'bl_stack.layer_list.2.b_conv']
num_param 55808
========starting phase 1/3==========
val metric init {'loss': 0.1476345956325531, 'loss_no_reg': 0.14376674592494965, 'corr': [0.058109296390085594, 0.02109747215932515, 0.027515191091892426, -0.03901109963146236, 0.01123947523614571, -0.05336567342002975, -0.00433439661308649, -0.01787438968481538, 0.03863201141127208, 0.15008972727755626, 0.023520704968540695, 0.1414776669269659, 0.08566432043736724, 0.029155014142259017, -0.03247472192507334, -0.029545185814865985, 0.0015026930543709923, 0.008137061881077714, 0.007465024719027984, 0.17541698514456022, 0.02855167940107918, -0.024510255505319403, -0.03355362851738701, 0.0415603074538, 0.024597719129324878, -0.0012241833990332763, -0.011557944100550114, 0.049707678594448926, 0.045988430414181665, -0.05450135989435649, -0.0036988108943219744, -0.05916807907626321, 0.0747614933806611

test metric {'loss': 0.12537304844175065, 'loss_no_reg': 0.12169387936592102, 'corr': [0.2640944698583386, 0.39853824833118345, 0.45451993854284034, 0.6945781093040657, 0.40965658030646035, 0.4846753140930752, 0.3012851611922463, 0.5069808098963553, 0.604526811899726, 0.6111520791415926, 0.5867179794380383, 0.39607203023578863, 0.503588709328948, 0.6043223743578908, 0.3252115388033378, 0.5233069907698216, 0.5833182637432823, 0.5259455059957385, 0.5100950576072043, 0.5279141245075993, 0.41974065521737786, 0.38763355466107674, 0.1567147118971369, 0.0929028895100969, 0.642274338816769, 0.4577207775371074, 0.365939869668403, 0.24906836112080463, 0.5691474649823973, 0.46873473816983163, 0.2833958909896383, 0.5178815113261437, 0.30132270630498026, 0.48846165444818757, 0.5393707888980279, 0.4592736838821383, 0.4968273609972808, 0.49701852722669393, 0.3662424239756368, 0.36309519868549095, 0.25685240429650513, 0.46997775831109595, 0.3388458359038314, 0.3441267788759555, 0.4154574471511596, 0.3

========starting epoch 500==========
500-0, train loss 0.11549799144268036
train loss 0.11549799144268036
val metric {'loss': 0.11965274065732956, 'loss_no_reg': 0.11553753912448883, 'corr': [0.2261752526667472, 0.4720380604592146, 0.5302207283444262, 0.7313928967661143, 0.41555725513911373, 0.4881337180353353, 0.45550017860825365, 0.5387738810347298, 0.6250514360946369, 0.6569412923164155, 0.6338948716077628, 0.44606272715264883, 0.5534456204641975, 0.6106806369664659, 0.5033404970663641, 0.5629554743027476, 0.6605597008533499, 0.5759892024722122, 0.5449549815420831, 0.6315815938982899, 0.44424920053945904, 0.4019351103268985, 0.20256743839087832, 0.1163372618008698, 0.7315470797790292, 0.5098829942261722, 0.4421866155115973, 0.3293831351737197, 0.6287313065363634, 0.5278769812769307, 0.2994225955577393, 0.5400140973516222, 0.4833036124901677, 0.494246068939426, 0.594145331231263, 0.49157741499482654, 0.5211582638008629, 0.5624799530679154, 0.41387317452947414, 0.38470730164680955, 0.

test metric {'loss': 0.11776485506977354, 'loss_no_reg': 0.11372369527816772, 'corr': [0.29709497611845426, 0.47089579446666374, 0.5664735505095151, 0.7334289549622994, 0.44171977852461525, 0.5585991751295571, 0.5118284481705428, 0.6083062686113916, 0.6536716503351345, 0.6899869802932843, 0.6603241814136084, 0.44720215231293003, 0.5513191133187365, 0.6536488530550445, 0.4953738433702479, 0.5742424775404227, 0.698282479413654, 0.6092399248573497, 0.5337009513003119, 0.6053023218001599, 0.472554461831702, 0.44682995413125054, 0.2145302476166825, 0.09316198363923303, 0.7134237954373198, 0.567077725409288, 0.48641403604294803, 0.3110143769170534, 0.6070957274388507, 0.5061521733347399, 0.3299119656799985, 0.5881601863812862, 0.43450140094299644, 0.5750049628837655, 0.6056286610862986, 0.5259040391767746, 0.5652503329570076, 0.5611666018230301, 0.40136661873212076, 0.39907111018276126, 0.36548543365674907, 0.5288511937606472, 0.43237236661605116, 0.41604813145626734, 0.5424044100302309, 0.3

========starting epoch 1000==========
1000-0, train loss 0.11705164611339569
train loss 0.11705164611339569
val metric {'loss': 0.11635163724422455, 'loss_no_reg': 0.11225222796201706, 'corr': [0.26115502959012743, 0.49175355375423263, 0.5790859098198946, 0.7347143016256874, 0.48242675903598486, 0.5086637230970547, 0.47027274729621615, 0.626066400366576, 0.656550995377327, 0.6914012350674132, 0.6645056590947398, 0.48767292345498786, 0.5717103830854339, 0.6266053884442369, 0.5692760319972376, 0.5831987554353173, 0.6823626045820093, 0.6033598328185181, 0.5523044381237812, 0.6563961177218078, 0.4735187109100071, 0.4244188878907167, 0.2459054298672036, 0.11501404018408753, 0.7367110561136636, 0.5610884918344548, 0.5241942218666671, 0.38160064841808583, 0.6463337015093007, 0.5352844151109815, 0.33166411641588667, 0.5830962782559074, 0.5356303545315015, 0.5615333448369358, 0.6290032176025527, 0.5090357804130023, 0.5787700705711032, 0.5749481964696863, 0.43866608695420306, 0.3864159127291078,

...

========starting epoch 300==========
300-0, train loss 0.10609088838100433
train loss 0.10609088838100433
val metric {'loss': 0.1139493703842163, 'loss_no_reg': 0.11024188995361328, 'corr': [0.2587007856581291, 0.49260368801591725, 0.5980820662196364, 0.7682757356454697, 0.5123388522908421, 0.5145038518039864, 0.5092216915723142, 0.6420010599084367, 0.6659540528662531, 0.7052051736738478, 0.6856275443755615, 0.5210619340165424, 0.6039872265454371, 0.6566449700615403, 0.6060938112814251, 0.6055986193986003, 0.6935324211252596, 0.6096473508325143, 0.5967063972245388, 0.688595863011753, 0.4780999394612501, 0.44573707437804216, 0.3671148554572965, 0.10771850209844691, 0.7477935242436577, 0.6015826683017975, 0.5492161077088272, 0.39316681967730216, 0.6714766772608485, 0.5464389638878187, 0.36459893819830075, 0.5829413607742536, 0.5263808091217057, 0.58650861600497, 0.6337617707659944, 0.5116754807309492, 0.6146318260298668, 0.6183947370410172, 0.47138393980985516, 0.39908808431996123, 0.386

test metric {'loss': 0.11562533463750567, 'loss_no_reg': 0.1118086576461792, 'corr': [0.32755021072280344, 0.509606486495397, 0.5985972316528211, 0.7634540587784541, 0.5074204357884706, 0.5834576293858831, 0.5302572628847961, 0.6272135116036959, 0.6739360277756821, 0.7107975742167782, 0.6956759959485335, 0.4713059564152645, 0.5959966986854943, 0.6721490054264503, 0.5820240130915508, 0.6114018104177549, 0.7197298803847583, 0.6419042392749422, 0.6030246596672759, 0.6682610329880991, 0.5059208383122569, 0.4633467483225867, 0.3240334730372007, 0.10363983184545242, 0.7457908113458402, 0.6068591546702786, 0.5370114519009852, 0.34765433104189625, 0.6420894672391788, 0.5278643856727245, 0.36426831217280686, 0.5929097356883039, 0.4486725441217666, 0.5989674086740613, 0.6265155288565626, 0.5393250567337045, 0.6322084892493514, 0.6079862148643844, 0.4508729853128798, 0.403810221965577, 0.421061406588489, 0.5608821484647725, 0.45861033547544783, 0.46135761660213315, 0.6310148838935418, 0.451341577

test metric {'loss': 0.1145750635436603, 'loss_no_reg': 0.11065530776977539, 'corr': [0.3274205136544549, 0.5082924814417211, 0.5991866503691072, 0.7612942438183246, 0.5043237106260434, 0.5805312887008068, 0.5315291388574823, 0.6303793285870372, 0.6740880698946917, 0.7070338155766015, 0.6933404090459204, 0.4717545642641444, 0.5955924630370479, 0.6691604763424812, 0.5686200892701686, 0.6125834066953975, 0.720329656445783, 0.6404793462041998, 0.6009388883642327, 0.6675259444965194, 0.5032729929822021, 0.4623074074839426, 0.3306431871957383, 0.10510107680915909, 0.7455249934903734, 0.6093394953373674, 0.5405802647071447, 0.34678475491652744, 0.6434103162391146, 0.5270200213169838, 0.3647121907008106, 0.5916971775729976, 0.4476087920370389, 0.6014743271027815, 0.6222104725371715, 0.537394727115849, 0.629461386563748, 0.6078865967692229, 0.44974036051244065, 0.4034705707230865, 0.4134359819518321, 0.5602932404608965, 0.45871160087787366, 0.4571008819143404, 0.6293543891592337, 0.45263295725

========starting epoch 400==========
400-0, train loss 0.10422135144472122
train loss 0.10422135144472122
val metric {'loss': 0.11375600546598434, 'loss_no_reg': 0.11018171906471252, 'corr': [0.255759262345406, 0.4930739057958955, 0.6002465367218832, 0.7651421887416534, 0.5114942186287571, 0.5150428223721665, 0.5045897209236782, 0.6509587787078803, 0.664764369473938, 0.7072705382026773, 0.688264253349347, 0.5209861256901361, 0.6026811092670847, 0.6572353143693331, 0.607243219211193, 0.6064911868783216, 0.6956592406161847, 0.6088714262729616, 0.5973174823723859, 0.6904987524388231, 0.4776016388992874, 0.4437319011677402, 0.36703183634788117, 0.10907666369574483, 0.748390223685092, 0.6037500911392146, 0.5600472080227015, 0.393340675593045, 0.6707275239389672, 0.5487003218502987, 0.36180681634619793, 0.5852667052996152, 0.5209145719090098, 0.5945896371108422, 0.6347672053331489, 0.5110330433840558, 0.6146021655247221, 0.6184035493257489, 0.47215688825350965, 0.3977510803172619, 0.38197427

test metric {'loss': 0.11483048647642136, 'loss_no_reg': 0.1109386757016182, 'corr': [0.3294093533535788, 0.5093959052504565, 0.600066643580512, 0.7643679334238298, 0.5089854636137634, 0.5827314258982443, 0.5308919435985806, 0.6301130965182367, 0.6743148456902663, 0.7098773267741918, 0.6959813285529892, 0.4717097622909127, 0.5957594035023419, 0.6706337021089852, 0.5790119293170148, 0.6134006027635808, 0.7200318989876173, 0.6422419774408124, 0.6018744576745585, 0.6679810954763218, 0.5041184542962783, 0.46240834524590946, 0.33173495249957, 0.10396752271012191, 0.7440340253548361, 0.6082628065574177, 0.5381500679149134, 0.34803598130851293, 0.6434924952566211, 0.5292376252589575, 0.36756842403844725, 0.5935065612893988, 0.4482519409301169, 0.6032035079826867, 0.6235042890988475, 0.5398882771003072, 0.6321379993574261, 0.6080497961433169, 0.45145802427602444, 0.40670884763397874, 0.41795888363117184, 0.5613986466801182, 0.45868563729550027, 0.45982833551140045, 0.6301020210118535, 0.453652